# 04. 답변의 형식을 컨트롤 하는 방법

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model ="deepseek-r1:8b")

In [2]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(
    template="what is the capital of{country}? Return only the city name.", # prompt 도 city 만 받도록 수정
    input_variables=["country"],
)

prompt = prompt_template.invoke({'country': '오스트레일리아'})

In [3]:
ai_message = llm.invoke(prompt)
ai_message

AIMessage(content='Canberra', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-11-10T23:24:16.135602Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4884938250, 'load_duration': 53547500, 'prompt_eval_count': 20, 'prompt_eval_duration': 942224375, 'eval_count': 159, 'eval_duration': 3848506092, 'model_name': 'deepseek-r1:8b', 'model_provider': 'ollama'}, id='lc_run--7e1b82c0-f519-4dc9-96fb-d0ef0d1fb333-0', usage_metadata={'input_tokens': 20, 'output_tokens': 159, 'total_tokens': 179})

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(ai_message)

'Canberra'

아래 방식은 JsonOutputParser 를 테스트하는건데, 생각보다 안될때가 많음

In [5]:
country_detail_prompt_template = PromptTemplate(
    template="""
    Return the detail country information of {country}.
    - Capital
    - Population
    - Language
    - Currency

    return it in JSON format. and return only the JSON object.
    """,
    input_variables=["country"],
)

country_detail_prompt = country_detail_prompt_template.invoke({'country': 'France'})

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

country_message = llm.invoke(country_detail_prompt)

In [7]:
country_message

AIMessage(content='{"Capital":"Paris","Population":67400000,"Language":"French","Currency":"Euro"}', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-11-10T23:24:41.884019Z', 'done': True, 'done_reason': 'stop', 'total_duration': 25722137583, 'load_duration': 43441500, 'prompt_eval_count': 43, 'prompt_eval_duration': 184921791, 'eval_count': 1068, 'eval_duration': 25331246091, 'model_name': 'deepseek-r1:8b', 'model_provider': 'ollama'}, id='lc_run--988c63a1-397a-4110-a2b3-eaf862e47c36-0', usage_metadata={'input_tokens': 43, 'output_tokens': 1068, 'total_tokens': 1111})

In [8]:
json_parser.invoke(country_message)

{'Capital': 'Paris',
 'Population': 67400000,
 'Language': 'French',
 'Currency': 'Euro'}

In [9]:
type(country_message.content)

str

pydantic 으로 model 을 선언해 llm 에 연결함으로 제어 가능

In [10]:
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital city of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

In [11]:
structured_llm = llm.with_structured_output(CountryDetail)

In [12]:
result_model =structured_llm.invoke(country_detail_prompt)

In [13]:
result_model

CountryDetail(capital='Paris', population=67415000, language='French', currency='Euro')

In [14]:
result_model.model_dump()

{'capital': 'Paris',
 'population': 67415000,
 'language': 'French',
 'currency': 'Euro'}